In [1]:
"""Accuracy 98.12%"""
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # one_hot True면 Y값을 one_hot으로 읽어옴

nb_classes = 10

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

"""
dropout은 네트웍이 overfitting되지 않게 학습을 시킬때 일부를 끊고 학습을 시키는 것이다.
"""
W1 = tf.get_variable("W1", shape=[784, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1) # Activation function으로 relu 사용.
# dropout이라는 레이어가 하나 더 있다고 생각을 하면 된다.
"""
여기서 몇프로를 끊어버릴 것인가 킵할 것인가를 정해줘야하는데
텐서플로 1.0에선 keep으로 변수명이 바뀜
전체에서 몇프로의 네트웍을 keep할 것인가를 정해준다.

통상적으로 train을 할 때는 0.5에서 0.7정도로 함
testing/실전에서는 반드시 1로 해야한다.

그래서 이렇게 하기위해서 
keep_prob를 보통 placeholder로 정하고
그리고 이걸 학습/테스팅할때 다르게 feed_dict를 통해 값을 넘겨주게 된다.
"""
keep_prob = tf.placeholder(tf.float32)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)


W3 = tf.get_variable("W3", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[512, 512],
                  initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


W5 = tf.get_variable("W5", shape=[512, 10],
                  initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5



# define cost/loss & optimizer
learning_rate = 0.001
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
"""
optimizer도 GradientDescent만 있는게 아니라 다양하게 있다.
Adam이라는 것이 상당히 좋은 결과를 만들어내기에 첨에 시작을 Adam으로 하는게 좋다.
뭐 잘안되면 바꿔보고,,
"""
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


# Test model
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1)) # one_hot인 Y
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# parameters
# 데이터가 크니깐 한꺼번에 할수없으니 조금씩 학습
# epoch: 몇갠지는 모르지만 전체 데이터셋을 한번 다 학습시킨것을 1 epoch
# batch_size: 전체 데이터를 한꺼번에 메모리에 못올릴때 짤라서 올리는데 그 size
# iterations: 전체 데이터가 1000개이고 batch_size가 500개 일때 1 epoch을 하기 위해선 2 iterations을 돌아야한다.
training_epochs = 15
batch_size = 100

with tf.Session() as sess: # 세션 열고
    # Initialize Tensorflow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(training_epochs): # 15번 돌아가겠지
        avg_cost = 0
        # 전체 사이즈를 batch_size로 나누면 iterations 수 구할 수있음
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch): # i가 iterations
            batch_xs, batch_ys = mnist.train.next_batch(batch_size) # 100개씩 읽어와서 학습을 함
            c, _ = sess.run([cost, optimizer], 
                            feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7}) # train할때 0.7
            avg_cost += c / total_batch
        
        print('Epoch: ', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
    # sess.run()으로 돌릴 수도 있고, accuracy.eval()로도 돌릴 수 있음
    print("Accuracy: ", sess.run(accuracy, feed_dict={
        X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1})) # testing시는 반드시 1로!!!!!


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch:  0001 cost = 0.509569593
Epoch:  0002 cost = 0.177544231
Epoch:  0003 cost = 0.137331789
Epoch:  0004 cost = 0.109875702
Epoch:  0005 cost = 0.096490713
Epoch:  0006 cost = 0.086236212
Epoch:  0007 cost = 0.075983599
Epoch:  0008 cost = 0.066496225
Epoch:  0009 cost = 0.067858918
Epoch:  0010 cost = 0.058273883
Epoch:  0011 cost = 0.055572274
Epoch:  0012 cost = 0.052144047
Epoch:  0013 cost = 0.050174136
Epoch:  0014 cost = 0.048355684
Epoch:  0015 cost = 0.046828789
Accuracy:  0.9812
